In [7]:
class sentiment:
       POSITIVE = "POSITIVE" 
       NEUTRAL = "NEUTRAL"
       NEGATIVE = "NEGATIVE"
class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    def get_sentiment(self):
        if self.score<=2:
            return sentiment.NEGATIVE
        elif self.score ==3:
            return sentiment.NEUTRAL
        else:  #scroe of 5 or 4
            return sentiment.POSITIVE


In [8]:
import pandas as pd
import json

In [9]:
file_name = "D:\\MY WORK\\Python\\Basic Project\\data\\Books_small.json"
df = pd.read_json(file_name, lines=True)
# Set display options
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)
df.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1E5ZR1Z4OQJG,1495329321,"Pure Jonel ""Pure Jonel""","[0, 0]",Da Silva takes the divine by storm with this u...,4,An amazing first novel,1396137600,"03 30, 2014"
1,A30PZPI6FPH0A7,0399157565,Jackmollie,"[0, 0]",For me personally it's the most disappointing ...,2,disappointed,1400112000,"05 15, 2014"
2,A1GQ2UI5BKCCRD,0984528105,Gail Hodges,"[0, 0]","Very simple book, but leaves you feeling good....",4,Good book,1401235200,"05 28, 2014"
3,A2DF4LQQI6KSQ2,0804139024,Olga,"[0, 0]",I read a library copy of this exceptionally we...,5,Science Fiction at its best!,1396483200,"04 3, 2014"
4,A1UAMAWY966P2,0765317583,Nadyne M Ichimura,"[0, 0]",With the government knowing this could happen ...,5,Excellent story,1397001600,"04 9, 2014"


In [10]:
# List to store the reviews
reviews = []

# Open the JSON file and read line by line
with open(file_name) as f:
    for line in f:
        review_data = json.loads(line)
        review_instance = Review(review_data['reviewText'], review_data['overall'])
        reviews.append(review_instance)

In [11]:
import random
class ReviewContainer():
  def __init__(self, reviews):
    self.reviews = reviews

  def get_text(self):
    return [x.text for x in self.reviews]

  def get_sentiment(self):
    return [x.sentiment for x in self.reviews]

  def evenly_distribute(self):
    negative = list(filter(lambda x: x.sentiment == sentiment.NEGATIVE, self.reviews))
    positive = list(filter(lambda x: x.sentiment == sentiment.POSITIVE, self.reviews))
    positive_shrunk = positive[:len(negative)]
    self.reviews = negative + positive_shrunk
    random.shuffle(self.reviews)

In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(reviews, test_size = 0.25, random_state = 42)

train = ReviewContainer(train)
test = ReviewContainer(test)

train.evenly_distribute()
test.evenly_distribute()

train_x = train.get_text()
train_y = train.get_sentiment()


test_x = test.get_text()
test_y = test.get_sentiment()

print(train_y.count(sentiment.POSITIVE))
print(test_y.count(sentiment.NEGATIVE))


51
11


In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vector = vectorizer.fit_transform(train_x)
test_x_vector = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vector[0])

Heaven&#8217;s CryBy: Charmaine GallowayThis story is about a teenage girl named Heaven. Heaven is a pastor&#8217;s daughter and her mother is the first lady of their church. She&#8217;s dealing with so much along with the everyday struggles of being a teenager.Heaven&#8217;s dealing with such as a teenager from being bullied at school to her sexuality. She feels she can&#8217;t talk to her parents because they always trying to keep the perfect image for the church. Her parents are always preaching and quoting Bible scriptures.Heaven has a best friend named Jordan who she has more than just friendly feelings for. She has no one to talk to about her feeling for Jordan. Her parent is so strict and live by the every word in the Bible that she&#8217;s afraid to even share her feelings. Her parents have tried to instill the Bible into her.When the bullies at school find out how she feels about Jordan they embarrass her during lunch. Heaven is so embarrassed they she starts fighting the bull

In [14]:
from sklearn import svm
from sklearn.svm import SVC

clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vector,train_y)

SVC(kernel='linear')

In [15]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vector,train_y)
#clf_dec.predict(train_x_vector[0])

DecisionTreeClassifier()

In [16]:
from sklearn.naive_bayes import GaussianNB
clf_gnb = GaussianNB()

dense_train_x = train_x_vector.toarray()
dense_test_x = test_x_vector.toarray()

clf_gnb.fit(dense_train_x,train_y)
#clf_gnb.predict(dense_train_x)

GaussianNB()

In [17]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vector,train_y)

LogisticRegression()

In [18]:
print(clf_svm.score(test_x_vector, test_y))
print(clf_dec.score(test_x_vector, test_y))
print(clf_gnb.score(dense_test_x, test_y))
print(clf_log.score(test_x_vector, test_y))

0.7272727272727273
0.6363636363636364
0.5
0.7727272727272727


In [19]:
from sklearn.metrics import f1_score

print(f1_score(test_y, clf_svm.predict(test_x_vector), average = None, labels=[sentiment.POSITIVE, sentiment.NEGATIVE]))
print(f1_score(test_y, clf_dec.predict(test_x_vector), average = None, labels=[sentiment.POSITIVE, sentiment.NEGATIVE]))
print(f1_score(test_y, clf_gnb.predict(dense_test_x), average = None, labels=[sentiment.POSITIVE, sentiment.NEGATIVE]))
print(f1_score(test_y, clf_log.predict(test_x_vector), average = None, labels=[sentiment.POSITIVE, sentiment.NEGATIVE]))

[0.75 0.7 ]
[0.69230769 0.55555556]
[0.42105263 0.56      ]
[0.7826087  0.76190476]


In [20]:
# Let's enter some input of our own
custom_input = ['The best investment of my life', "Bad book, absolutely horrible", "You won't regret purchasing it", "It was truly a delight reading this book", "this book was beyond amazing! the best murder mystery book i have EVER read, the suspects are never too obvious and nor are they out of the blue, with the way holly jackson writes, you'd feel like you yourself were investigating! she writes so realistically, the characters feel so life-like, every character is understandable and the plot is marvellous, cant wait to get the rest of the series and i know i wont be disappointed. i have never gasped out loud to a book so many times but here we are with this book, i cant begin to describe how good it was, you think its over, you think everything's cooled down and solved and then bam, another story opens up. love love love this book, definitely recommend."]
# Lets convert these inputs to numbers so that our model can learn
custom_input_vectors = vectorizer.transform(custom_input)

print(clf_svm.predict(custom_input_vectors))
print(clf_log.predict(custom_input_vectors))
print(clf_dec.predict(custom_input_vectors))

['POSITIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE' 'POSITIVE']
['POSITIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE' 'POSITIVE']
['NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE']


In [21]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Lets setup grid search CV for SVM
parameters = {"kernel": ("linear", "rbf"), "C": (1, 4, 8, 16, 32)}
clf_svm = svm.SVC()
best_clf_svm = GridSearchCV(clf_svm, parameters, cv = 5)
best_clf_svm.fit(train_x_vector, train_y)

# Now since we know what are the best parameters, lets have a look at them
best_params = best_clf_svm.best_params_
print("Best Parameters: ", best_params)

# Now let's see if the accuracy has improved or not
test_predictions = best_clf_svm.predict(test_x_vector)
accuracy = accuracy_score(test_y, test_predictions)
print("Accuracy on Test Set:", accuracy)

Best Parameters:  {'C': 4, 'kernel': 'rbf'}
Accuracy on Test Set: 0.7727272727272727


In [22]:
# Now lets setup grid search CV for Logistic Regression model
# We will start by intializing our model
clf_log = LogisticRegression()

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # regularization parameter
    'penalty': ['l2']  # lbfgs solver supports only 'l2' or 'none'
}


clf_log = GridSearchCV(clf_log, param_grid, cv = 5, scoring="accuracy")

clf_log.fit(train_x_vector, train_y)

print("Best Hyperparameters: ", clf_log.best_params_)

best_clf_log = clf_log.best_estimator_

predictions = best_clf_log.predict(test_x_vector)

accuracy = accuracy_score(test_y, predictions)
print("Accuracy on test set: ", format(accuracy * 100))

Best Hyperparameters:  {'C': 10, 'penalty': 'l2'}
Accuracy on test set:  72.72727272727273


In [23]:
# Let's enter some input of our own
custom_input = ['The best investment of my life', "Bad book, absolutely horrible", "You won't regret purchasing it", "It was truly a delight reading this book", "this book was beyond amazing! the best murder mystery book i have EVER read, the suspects are never too obvious and nor are they out of the blue, with the way holly jackson writes, you'd feel like you yourself were investigating! she writes so realistically, the characters feel so life-like, every character is understandable and the plot is marvellous, cant wait to get the rest of the series and i know i wont be disappointed. i have never gasped out loud to a book so many times but here we are with this book, i cant begin to describe how good it was, you think its over, you think everything's cooled down and solved and then bam, another story opens up. love love love this book, definitely recommend."]
# Lets convert these inputs to numbers so that our model can learn
custom_input_vectors = vectorizer.transform(custom_input)

print(best_clf_log.predict(custom_input_vectors))

['POSITIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE' 'POSITIVE']


SAVING MODEL

In [25]:
import pickle

with open('D:\\MY WORK\\Python\\models\\sentiment_clf.pkl','wb') as f:
    pickle.dump(best_clf_log,f)  # dump the classifier into the file

LOADING MODEL

In [26]:
with open('D:\\MY WORK\\Python\\models\\sentiment_clf.pkl','rb') as f:
    loaded_clf = pickle.load(f)

In [28]:
print(test_x[9])

loaded_clf.predict(test_x_vector[7])

I think the book could have been great if the author could have decided what kind of book she wanted to write.  She has awesome descriptions, too vivid personal parts, great informational parts, poorly organized presentation.  I understood what she was aiming for, but the journey was not fun.  Will not read her again!


array(['NEGATIVE'], dtype='<U8')